In [1]:
# https://msprime.readthedocs.io/en/stable/tutorial.html#demography

In [56]:
from math import (exp, log)
import numpy as np
import msprime

In [85]:
RHO_HUMAN = 1.6*10e-9
MU_HUMAN = 1.25*10e-8
RHO_LIMIT = (log(RHO_HUMAN)-100, log(RHO_HUMAN)+100)
MU_LIMIT = (log(MU_HUMAN)-100, log(MU_HUMAN)+100)

LENGTH_NORMALIZE_CONST = 4
ZIPPED = False
NUMBER_OF_EVENTS_LIMITS = (1, 20)
MAX_T_LIMITS = (0.01, 30)
LAMBDA_EXP = 1.0
POPULATION_LIMITS = (250, 100000)
POPULATION = 5000

IS_SEMMETRIC = True

N = 20

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [18]:
population_configurations = [
    msprime.PopulationConfiguration(
        sample_size=1, initial_size=100),
    msprime.PopulationConfiguration(
        sample_size=1, initial_size=200, growth_rate=-0.01)
]
# What's wrong with growth_rate? Why population does growth only if it's negative?

In [39]:
def create_N_population(populations_parametrs: dict) -> list:
    """
    population_parametrs: {population_name:int : [sample_size:int, initial_size:float, growth_rate:float]}
    """
    populations_configuration = []
    for population_name, parametrs in populations_parametrs.items():
        if len(parametrs)  == 3:
            sample_size, initial_size, growth_rate = parametrs
        elif len(parametrs)  == 2:
            sample_size, initial_size = parametrs
        else:
            raise f"Number ov parametrs error: expected 2 or 3, but got {len(parametrs)}"
        populations_configuration.append(
            msprime.PopulationConfiguration(
                sample_size=sample_size, 
                initial_size=initial_size, 
                growth_rate=growth_rate)
        )
    return populations_configuration

In [51]:
def create_random_migration_matrix(N: int, max_migration_rate, is_semmetric = IS_SEMMETRIC):
    migration_matrix = np.zeros(shape=(N,N))
    for i in range(N):
        for j in range(i + 1,N):
            migration_matrix[i][j] = np.random.random()*max_migration_rate
            if is_semmetric:
                migration_matrix[j][i] = migration_matrix[i][j]
            else:
                migration_matrix[j][i] = np.random.random()*max_migration_rate
    return migration_matrix

    demographic_events = [
        # CEU and CHB merge into B with rate changes at T_EU_AS
        
        msprime.MassMigration(
         time=T_EU_AS, source=2, destination=1, proportion=1.0)
            ,
        msprime.MigrationRateChange(time=T_EU_AS, rate=0),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EU_AS, initial_size=N_B, growth_rate=0, population_id=1),
        # Population B merges into YRI at T_B
        msprime.MassMigration(
            time=T_B, source=1, destination=0, proportion=1.0),
        msprime.MigrationRateChange(time=T_B, rate=0),
        # Size changes to N_A at T_AF
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]

In [119]:
MAX_MIGRATION_RATE = 10

def create_MassMigration(time: float, number_of_populations: int, populations: list) -> list:
    if number_of_populations == 1:
        raise "Only one population"
    source, destination = 0,0
    while source == destination:
        source, destination = np.random.randint(number_of_populations,size=2)
    proportion = min(np.random.rand(), 1.0) # learn max of np.random.rand()
    return [msprime.MassMigration(time=time,source=source,destination=destination, proportion=proportion)]

def create_MigrationRateChange(time: float, number_of_populations: int, populations: list) -> list:
    rate = np.random.random()*MAX_MIGRATION_RATE
    i,j = np.random.randint(low=number_of_populations,size=(2))
    while i == j:
        i,j = np.random.randint(low=number_of_populations,size=(2))
    migration = [msprime.MigrationRateChange(time=time,rate=rate,matrix_index=(i,j))]
    if IS_SEMMETRIC:
        migration.append(msprime.MigrationRateChange(time=time,rate=rate,matrix_index=(j,i)))
    return migration
    
def create_PopulationParametersChange(time: float, number_of_populations: int, populations: list) -> list:
    population_id = np.random.randint(number_of_populations)
    size = int(np.random.beta(a=2, b=5) * populations[population_id])
    while size <= 0:
        size = int(np.random.beta(a=2, b=5) * populations[population_id])
    return [msprime.PopulationParametersChange(time=time, initial_size=size, population_id=population_id)]

events = {
    "MassMigration": create_MassMigration,
    "MigrationRateChange": create_MigrationRateChange,
    "PopulationParametersChange": create_PopulationParametersChange,
}

events = [create_MassMigration,create_MigrationRateChange,create_PopulationParametersChange]

def create_demographic_events(number_of_populations: int, populations: list) -> list:
    """
    """
    low, high = NUMBER_OF_EVENTS_LIMITS
    number_of_events = np.random.randint(low=low, high=high)
    times = sorted(np.random.exponential(LAMBDA_EXP, size=number_of_events))
    low, high = MAX_T_LIMITS
    max_t = np.random.uniform(low=low, high=high)
    
    alpha = 1.0
    beta = np.log(max_t + 1)/times[-1]

    def to_exp_time(time: float) -> float:
        # time -> exponentional time
        return alpha*(np.exp(beta*time) - 1)
    
    exp_times = [to_exp_time(t) for t in times]
    
    demographic_events = []
    for time in exp_times:
        event = np.random.choice(events)(time, number_of_populations, populations)
        for ev in event:
            demographic_events.append(ev)
    return demographic_events

In [120]:
create_demographic_events(2,[10,10])

[{'type': 'mass_migration', 'time': 0.007315565144512037, 'source': 1, 'dest': 0, 'proportion': 0.08289132169335423},
 {'type': 'mass_migration', 'time': 0.03768476784689545, 'source': 1, 'dest': 0, 'proportion': 0.9632225394406113},
 {'type': 'mass_migration', 'time': 0.057237492074648344, 'source': 1, 'dest': 0, 'proportion': 0.04315991195057611},
 {'type': 'mass_migration', 'time': 0.07815105517674015, 'source': 1, 'dest': 0, 'proportion': 0.2529901679892411},
 {'type': 'population_parameters_change', 'time': 0.08409354322784646, 'growth_rate': None, 'initial_size': 2, 'population': 1},
 {'type': 'mass_migration', 'time': 0.3516277799970813, 'source': 0, 'dest': 1, 'proportion': 0.5843143119231002},
 {'type': 'population_parameters_change', 'time': 0.42245538966050744, 'growth_rate': None, 'initial_size': 2, 'population': 0},
 {'type': 'migration_rate_change', 'time': 0.4972899073092327, 'rate': 1.881211597237613, 'matrix_index': (1, 0)},
 {'type': 'migration_rate_change', 'time': 0

In [121]:
np.random.randint(2)

0

In [122]:
create_random_migration_matrix(3,1,is_semmetric=False)

array([[0.        , 0.37673897, 0.39298945],
       [0.7495783 , 0.        , 0.56908147],
       [0.82916422, 0.06351183, 0.        ]])

In [123]:
dd = msprime.DemographyDebugger(
    population_configurations=population_configurations,
    migration_matrix=create_random_migration_matrix(2,0.001),
    demographic_events=create_demographic_events(2,[10,10])
)
dd.print_history()

Model =  hudson(reference_size=1)
Epoch: 0 -- 2.2700182199703445 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |   100      100                0 |     0    3.68e-05 
1 |   200      205            -0.01 | 3.68e-05     0    

Events @ generation 2.2700182199703445
   - Mass migration: Lineages moved with probability 0.9131657522576428 backwards in time with source 0 & dest 1
                     (equivalent to migration from 1 to 0 forwards in time)
Epoch: 2.2700182199703445 -- inf generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |   100      100                0 |     0    3.68e-05 
1 |   205      inf            -0.01 | 3.68e-05     0    



In [151]:
generator = msprime.simulate(
        #sample_size=2,
        recombination_rate=RHO_HUMAN,
        mutation_rate=0.1,
        random_seed=42,
        model="hudson",
        length=100000,
        num_replicates=1,
        population_configurations=population_configurations,
        migration_matrix=create_random_migration_matrix(2,0.001),
        demographic_events=create_demographic_events(2,[10,10])
)

In [152]:
tr = []
for i in generator:
    tr.append(i)

In [ ]:
for variant in tr[0].variants():
    print(
        variant.site.id, variant.site.position,
        variant.alleles, variant.genotypes, sep="\t")

0	0.05187466740608215	('0', '1')	[1 0 0 1]
1	0.22265594452619553	('0', '1')	[0 1 1 0]
2	0.2275221049785614	('0', '1')	[1 0 0 0]
3	0.33443793654441833	('0', '1')	[0 1 1 0]
4	0.5590729415416718	('0', '1')	[0 0 0 1]
5	0.7044291123747826	('0', '1')	[1 0 0 1]
6	0.8427537977695465	('0', '1')	[1 0 0 0]
7	0.848621129989624	('0', '1')	[0 1 1 0]
8	0.870460644364357	('0', '1')	[0 1 1 0]
9	0.8793780580163002	('0', '1')	[0 1 1 0]
10	0.8963281288743019	('0', '1')	[0 1 1 0]
11	0.9291339665651321	('0', '1')	[0 0 1 0]
12	1.0910909622907639	('0', '1')	[0 0 0 1]
13	1.1635944247245789	('0', '1')	[0 1 0 0]
14	1.4609890058636665	('0', '1')	[0 1 1 0]
15	1.5771947801113129	('0', '1')	[1 0 0 0]
16	1.6734004020690918	('0', '1')	[1 0 0 0]
17	1.7405254766345024	('0', '1')	[0 1 1 0]
18	1.827208325266838	('0', '1')	[0 0 0 1]
19	2.010189928114414	('0', '1')	[0 1 1 0]
20	2.0513776689767838	('0', '1')	[1 0 0 0]
21	2.0888401195406914	('0', '1')	[0 0 0 1]
22	2.150982618331909	('0', '1')	[1 0 0 1]
23	2.1599233150482178	(

290	50.970748998224735	('0', '1')	[0 0 0 1]
291	51.23009905219078	('0', '1')	[1 0 0 1]
292	51.52022931724787	('0', '1')	[1 0 0 1]
293	51.832012832164764	('0', '1')	[0 0 0 1]
294	51.84737965464592	('0', '1')	[0 0 0 1]
295	52.23893094807863	('0', '1')	[1 0 0 0]
296	52.24691703915596	('0', '1')	[0 1 1 0]
297	52.58719902485609	('0', '1')	[0 1 1 0]
298	52.70864348858595	('0', '1')	[1 0 0 0]
299	52.725011482834816	('0', '1')	[0 0 0 1]
300	52.78419703245163	('0', '1')	[1 0 0 0]
301	52.80191544443369	('0', '1')	[0 1 1 0]
302	52.9599841684103	('0', '1')	[0 1 1 0]
303	53.09998523443937	('0', '1')	[0 1 1 0]
304	53.19889169186354	('0', '1')	[1 0 0 1]
305	53.24389785528183	('0', '1')	[0 1 1 0]
306	53.92194725573063	('0', '1')	[1 0 0 0]
307	54.19151857495308	('0', '1')	[0 1 1 0]
308	54.27026189863682	('0', '1')	[1 0 0 0]
309	54.72949706017971	('0', '1')	[0 1 1 0]
310	54.816994816064835	('0', '1')	[1 0 0 1]
311	55.00575061887503	('0', '1')	[0 0 0 1]
312	55.05313165485859	('0', '1')	[0 1 1 0]
313	55.6

838	156.00803308188915	('0', '1')	[0 0 0 1]
839	156.02563507854939	('0', '1')	[0 0 0 1]
840	156.24971129000187	('0', '1')	[1 0 0 0]
841	156.47571999579668	('0', '1')	[0 1 1 0]
842	156.51003923267126	('0', '1')	[0 1 0 0]
843	156.55006282031536	('0', '1')	[0 1 1 0]
844	156.5546728670597	('0', '1')	[1 0 0 1]
845	156.68494161218405	('0', '1')	[0 1 1 0]
846	157.13521279394627	('0', '1')	[0 0 0 1]
847	157.38008078187704	('0', '1')	[1 0 0 1]
848	157.84441493451595	('0', '1')	[1 0 0 1]
849	158.13421923667192	('0', '1')	[0 1 1 0]
850	158.68088230490685	('0', '1')	[0 1 1 0]
851	158.71317591518164	('0', '1')	[1 0 0 1]
852	159.27264466881752	('0', '1')	[0 0 0 1]
853	159.31942034512758	('0', '1')	[0 0 0 1]
854	159.47897918522358	('0', '1')	[0 1 1 0]
855	159.56114511936903	('0', '1')	[1 0 0 1]
856	159.59776937961578	('0', '1')	[0 1 1 0]
857	160.03616619855165	('0', '1')	[0 0 0 1]
858	160.27719248086214	('0', '1')	[0 0 1 0]
859	160.3755634278059	('0', '1')	[0 0 0 1]
860	160.46485397964716	('0', '1')	

1101	205.20433317869902	('0', '1')	[1 0 0 0]
1102	205.2668947726488	('0', '1')	[0 1 1 0]
1103	205.342099070549	('0', '1')	[0 0 0 1]
1104	205.35257644951344	('0', '1')	[0 1 1 0]
1105	206.1993582174182	('0', '1')	[0 1 1 0]
1106	206.23407326638699	('0', '1')	[1 0 0 1]
1107	206.37009292840958	('0', '1')	[1 0 0 0]
1108	206.52427338063717	('0', '1')	[1 0 0 0]
1109	206.56289998441935	('0', '1')	[0 0 0 1]
1110	206.85368217527866	('0', '1')	[0 0 0 1]
1111	206.86022471636534	('0', '1')	[1 0 0 0]
1112	206.88867662101984	('0', '1')	[1 0 0 0]
1113	206.97012078016996	('0', '1')	[1 0 0 1]
1114	207.14255515486002	('0', '1')	[1 0 0 0]
1115	207.84421358257532	('0', '1')	[1 0 0 0]
1116	208.13725423067808	('0', '1')	[0 1 1 0]
1117	208.3870116621256	('0', '1')	[0 1 1 0]
1118	208.4637526422739	('0', '1')	[0 1 1 0]
1119	208.50056316703558	('0', '1')	[0 1 1 0]
1120	208.7465487420559	('0', '1')	[0 1 1 0]
1121	208.95185880362988	('0', '1')	[0 1 1 0]
1122	208.98189395666122	('0', '1')	[0 0 0 1]
1123	209.21106915

1475	280.5735217407346	('0', '1')	[0 1 1 0]
1476	280.7412063702941	('0', '1')	[1 0 0 1]
1477	280.8935008943081	('0', '1')	[1 0 0 0]
1478	281.0116158798337	('0', '1')	[0 1 1 0]
1479	281.3665196299553	('0', '1')	[1 0 0 1]
1480	281.6105028614402	('0', '1')	[0 1 1 0]
1481	281.62768576294184	('0', '1')	[1 0 0 1]
1482	281.6398860886693	('0', '1')	[1 0 0 1]
1483	281.93092439323664	('0', '1')	[0 1 1 0]
1484	281.9679444655776	('0', '1')	[0 1 1 0]
1485	282.4818482622504	('0', '1')	[0 1 1 0]
1486	282.5327683240175	('0', '1')	[0 1 1 0]
1487	282.64387510716915	('0', '1')	[1 0 0 1]
1488	282.6739801093936	('0', '1')	[0 1 1 0]
1489	282.6974028721452	('0', '1')	[0 1 1 0]
1490	283.1279532983899	('0', '1')	[0 1 1 0]
1491	283.2242054864764	('0', '1')	[1 0 0 1]
1492	283.25882740318775	('0', '1')	[1 0 0 1]
1493	283.5351275280118	('0', '1')	[1 0 0 1]
1494	284.2883113771677	('0', '1')	[1 0 0 0]
1495	284.5190465450287	('0', '1')	[0 0 0 1]
1496	284.67944357544184	('0', '1')	[1 0 0 0]
1497	284.9226351827383	('0'

...

16193	3062.081290408969	('0', '1')	[1 0 0 0]
16194	3062.113793566823	('0', '1')	[0 1 1 0]
16195	3062.277799472213	('0', '1')	[0 1 1 0]
16196	3062.2970312833786	('0', '1')	[0 1 1 0]
16197	3062.3400351032615	('0', '1')	[0 0 0 1]
16198	3062.4389415606856	('0', '1')	[1 0 0 0]
16199	3062.569862231612	('0', '1')	[0 1 1 0]
16200	3063.190612010658	('0', '1')	[1 0 0 1]
16201	3063.2485635578632	('0', '1')	[0 1 1 0]
16202	3063.443396240473	('0', '1')	[1 0 0 1]
16203	3063.5102186352015	('0', '1')	[0 0 0 1]
16204	3063.5219300165772	('0', '1')	[0 1 1 0]
16205	3063.565935008228	('0', '1')	[1 0 0 1]
16206	3063.569637015462	('0', '1')	[0 0 1 0]
16207	3063.600161112845	('0', '1')	[1 0 0 1]
16208	3063.82505223155	('0', '1')	[1 0 0 0]
16209	3064.5789112895727	('0', '1')	[0 1 1 0]
16210	3064.7631036117673	('0', '1')	[0 1 1 0]
16211	3064.89912327379	('0', '1')	[1 0 0 0]
16212	3064.933232963085	('0', '1')	[1 0 0 0]
16213	3065.0062020868063	('0', '1')	[1 0 0 1]
16214	3065.1388224214315	('0', '1')	[0 1 1 0]
16

16596	3136.052726767957	('0', '1')	[0 1 1 0]
16597	3136.092098429799	('0', '1')	[1 0 0 0]
16598	3136.135218665004	('0', '1')	[0 1 1 0]
16599	3136.2979440018535	('0', '1')	[1 0 0 0]
16600	3136.384766548872	('0', '1')	[0 1 1 0]
16601	3136.6220908239484	('0', '1')	[1 0 0 0]
16602	3136.650030501187	('0', '1')	[0 1 1 0]
16603	3136.7348739877343	('0', '1')	[0 1 1 0]
16604	3136.7540825158358	('0', '1')	[1 0 0 0]
16605	3136.830567382276	('0', '1')	[0 1 1 0]
16606	3136.929520405829	('0', '1')	[0 1 1 0]
16607	3137.211059220135	('0', '1')	[1 0 0 1]
16608	3137.2745987027884	('0', '1')	[0 1 1 0]
16609	3137.4293379485607	('0', '1')	[0 1 1 0]
16610	3137.484029866755	('0', '1')	[1 0 0 0]
16611	3137.6575119793415	('0', '1')	[1 0 0 0]
16612	3137.968759983778	('0', '1')	[0 0 1 0]
16613	3138.5155161842704	('0', '1')	[0 1 1 0]
16614	3138.6005925014615	('0', '1')	[0 0 1 0]
16615	3138.623549602926	('0', '1')	[0 1 1 0]
16616	3138.6438058689237	('0', '1')	[0 0 0 1]
16617	3139.1868367791176	('0', '1')	[0 0 0 1]

16992	3209.2578941956162	('0', '1')	[0 1 1 0]
16993	3209.3842048197985	('0', '1')	[0 1 1 0]
16994	3209.4514928758144	('0', '1')	[0 0 1 0]
16995	3209.739178419113	('0', '1')	[1 0 0 0]
16996	3209.9869567900896	('0', '1')	[0 0 0 1]
16997	3210.4842364788055	('0', '1')	[1 0 0 0]
16998	3210.5663558468223	('0', '1')	[1 0 0 0]
16999	3210.712759755552	('0', '1')	[0 0 1 0]
17000	3210.957534611225	('0', '1')	[0 1 0 0]
17001	3210.999653674662	('0', '1')	[0 1 1 0]
17002	3211.2208660691977	('0', '1')	[1 0 0 0]
17003	3211.23288013041	('0', '1')	[0 1 1 0]
17004	3211.65903005749	('0', '1')	[0 1 1 0]
17005	3211.6984482854605	('0', '1')	[0 0 0 1]
17006	3211.8209870532155	('0', '1')	[0 1 1 0]
17007	3211.97547018528	('0', '1')	[0 1 0 0]
17008	3212.1982192620635	('0', '1')	[0 0 0 1]
17009	3212.257195264101	('0', '1')	[1 0 0 1]
17010	3212.5669298693538	('0', '1')	[1 0 0 1]
17011	3212.676662951708	('0', '1')	[0 1 1 0]
17012	3212.97247428447	('0', '1')	[1 0 0 1]
17013	3213.020064868033	('0', '1')	[1 0 0 0]
170

17407	3282.132768072188	('0', '1')	[0 1 1 0]
17408	3282.1926753968	('0', '1')	[0 0 0 1]
17409	3282.2669949382544	('0', '1')	[1 0 0 0]
17410	3282.3606627061963	('0', '1')	[1 0 0 1]
17411	3282.5494883581996	('0', '1')	[0 1 0 0]
17412	3283.1362448632717	('0', '1')	[0 1 1 0]
17413	3283.1612741574645	('0', '1')	[1 0 0 1]
17414	3283.41166023165	('0', '1')	[0 1 1 0]
17415	3283.4197161719203	('0', '1')	[0 1 1 0]
17416	3283.644956536591	('0', '1')	[0 0 0 1]
17417	3283.8432816788554	('0', '1')	[0 0 0 1]
17418	3283.9841907843947	('0', '1')	[0 1 1 0]
17419	3284.2566492035985	('0', '1')	[1 0 0 1]
17420	3284.488362260163	('0', '1')	[1 0 0 0]
17421	3284.735116176307	('0', '1')	[0 1 1 0]
17422	3284.7470371052623	('0', '1')	[0 1 1 0]
17423	3284.854185767472	('0', '1')	[1 0 0 0]
17424	3284.9545357748866	('0', '1')	[0 0 0 1]
17425	3285.00356990844	('0', '1')	[1 0 0 1]
17426	3285.183501429856	('0', '1')	[0 1 1 0]
17427	3285.213722847402	('0', '1')	[1 0 0 1]
17428	3285.347600467503	('0', '1')	[0 1 1 0]
174

17825	3356.755105778575	('0', '1')	[0 0 0 1]
17826	3357.6875925064087	('0', '1')	[1 0 0 0]
17827	3357.8128553926945	('0', '1')	[0 0 0 1]
17828	3357.8937873244286	('0', '1')	[1 0 0 0]
17829	3359.002131037414	('0', '1')	[1 0 0 1]
17830	3359.184693545103	('0', '1')	[0 1 1 0]
17831	3359.341202303767	('0', '1')	[1 0 0 0]
17832	3359.3443455174565	('0', '1')	[0 0 0 1]
17833	3359.5074201002717	('0', '1')	[0 1 1 0]
17834	3359.6037421375513	('0', '1')	[1 0 0 0]
17835	3359.7053959965706	('0', '1')	[1 0 0 1]
17836	3359.8229521885514	('0', '1')	[0 1 1 0]
17837	3359.966352581978	('0', '1')	[1 0 0 1]
17838	3359.97452493757	('0', '1')	[0 1 1 0]
17839	3360.0819995626807	('0', '1')	[0 0 0 1]
17840	3360.284329392016	('0', '1')	[0 1 1 0]
17841	3360.668569803238	('0', '1')	[0 0 0 1]
17842	3360.966476611793	('0', '1')	[0 0 1 0]
17843	3361.3434759899974	('0', '1')	[1 0 0 0]
17844	3361.360519193113	('0', '1')	[0 1 0 0]
17845	3361.3815205171704	('0', '1')	[1 0 0 0]
17846	3361.4485757425427	('0', '1')	[0 1 1 0]

In [82]:
i,j = np.random.randint(low=2,size=(2))

In [83]:
i,j

(1, 0)

In [84]:
a = []
b = [1,2]
a.append(*b)

TypeError: append() takes exactly one argument (2 given)